In [1]:
import numpy as np
import sklearn as sk
import pandas as pd
import sqlite3

In [3]:
nfl2016 = pd.read_csv('data/pbp-2016.csv')

In [4]:
var_list = ['GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'Description',
       'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsSack']

nfl2016 = nfl2016[var_list]

In [5]:
nfl2016 = nfl2016.dropna(subset=['OffenseTeam'])

In [6]:
def feat_pass_yrds(row):
    return row.IsPass * row.Yards

def feat_rush_yrds(row):
    return row.IsRush * row.Yards

def feat_pass_tds(row):
    return row.IsPass * row.IsTouchdown

def feat_rush_tds(row):
    return row.IsRush * row.IsTouchdown

In [7]:
nfl2016['PassYrds'] = nfl2016.apply(feat_pass_yrds, axis=1)
nfl2016['RushYrds'] = nfl2016.apply(feat_rush_yrds, axis=1)
nfl2016['PassTDs'] = nfl2016.apply(feat_pass_tds, axis=1)
nfl2016['RushTDs'] = nfl2016.apply(feat_rush_tds, axis=1)

In [8]:
off_vars = ['SeasonYear', 'Yards', 'IsRush', 'IsPass', 'IsIncomplete', 
            'PassYrds', 'RushYrds', 'PassTDs', 'RushTDs', 'IsTouchdown']

teams_off_counts = nfl2016.groupby('OffenseTeam').size()
teams_off = nfl2016.groupby('OffenseTeam')
teams_off = pd.DataFrame(teams_off.agg({'SeasonYear': 'first', 'Yards': 'sum', 'IsRush': 'sum', 'IsPass': 'sum', 
                                        'IsIncomplete': 'sum', 'PassYrds': 'sum', 'RushYrds': 'sum', 'PassTDs': 'sum', 
                                        'RushTDs': 'sum', 'IsTouchdown': 'sum'})[off_vars])
teams_off['Plays'] = teams_off_counts
teams_off.reset_index(level=0, inplace=True)
teams_off = teams_off.rename(columns={'OffenseTeam': 'Team', 'SeasonYear': 'Season', 'IsRush': 'Rushes', 'IsPass': 'Passes', 
                                      'IsIncomplete': 'Incompletes', 'IsTouchdown': 'TDs'})

In [9]:
teams_off.head()

,Team,Season,Yards,Rushes,Passes,Incompletes,PassYrds,RushYrds,PassTDs,RushTDs,TDs,Plays
0,ARI,2016,5803,365,647,255,4487,1627,27,19,50,1324
1,ATL,2016,6456,393,526,159,4847,1865,36,19,55,1240
2,BAL,2016,5789,345,658,222,4478,1551,21,9,31,1311
3,BUF,2016,5695,458,472,180,3245,2647,18,29,47,1250
4,CAR,2016,5718,426,563,252,4215,1760,26,16,43,1293
